In [ ]:
# Import libraries
import requests
import pandas as pd
import numpy as np
import time
import json
import sys
import os

In [ ]:
# Access token
Token = '...'  # Unique token required
header = {'token': Token}

In [ ]:
# Set up parameters for request url (for /data)
stationid = input('Enter valid station ID: ')  # Ex. West Palm Beach Airport Station (GHCND:USW00012844)
datasetid = input('Enter valid Dataset ID: ')  # Ex, GHCND, GSOM, GSOY...
datatypeid = input('Enter valid Datatype ID: ')  #Ex. TAVG, PRCP, AWND...
limit = input('Enter number of results: ')  # Maximum of 1000 results
startdate = input('Enter desired start date: ')  # ISO format YYYY-MM-DD
enddate = input('Enter desired end date: ')

In [ ]:
# Make station name recognizable
if stationid == 'GHCND:USW00012844':
    cur_loc = 'WPBA'  # West Palm Beach Airport (FL)
elif stationid == 'GHCND:USW00014732':
    cur_loc = 'LGA'  # La Guardia Airport (NY)
elif stationid == 'GHCND:USW00023174':
    cur_loc = 'LAX'  # Los Angeles International Airport (CA)

In [ ]:
# Generate url
url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid='+ datasetid +'&datatypeid='+ datatypeid +'&stationid='+ stationid +'&startdate='+ startdate +'&enddate='+ enddate +'&limit='+ limit

In [ ]:
try:
    # Request data
    response = requests.get(url, headers = header)
    
    # Determine if response is appropriate
    response.raise_for_status()
except requests.exceptions.HTTPError as error:
    print('Invalid response: {}'.format(error.response.status_code))
    sys.exit(1)

In [ ]:
# Retrieve JSON content
data = json.loads(response.text)

In [ ]:
values = list()
dates = list()
for i in data['results']:
    if i['datatype'] == datatypeid:
        if datatypeid == 'TAVG' or 'TMAX' or 'TMIN':
            # Original temp in tenths of degrees celsius
            value = round(((i['value']/10) * (9/5)) + 32, 2)
        else:  # Change for other datatypes
            value = i['value']
        # Reformat given date
        date = time.strftime('%d %b %y', time.strptime(i['date'], '%Y-%m-%dT%H:%M:%S'))
        # Populate lists for dates and values
        dates.append(date)
        values.append(value)

In [ ]:
# Creating Pandas dataframe
df_data = {datatypeid:values,
           'Date':dates}
df = pd.DataFrame(df_data)

# Add location column to dataframe
location = list()
for counter in range(0, len(df)):
    location.append(cur_loc)
df['Location'] = location
    
display(df[0:5])

In [ ]:
# Save data to csv
path = r'./data/'+ datatypeid +'_Summer_'+ cur_loc + '_' + time.strftime('%y', time.strptime(startdate, '%Y-%m-%d')) +'.csv'
df.to_csv(path, index = False)